In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.7 MB/s eta 0:00:00


In [3]:
import random
import os
import re
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = '/content/drive/MyDrive/DACON 경진대회/한솔 도배하자 질의응답/최종 제출/data/'
SEED = 42
reset_seeds(SEED)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# model load

In [4]:
from transformers import GPT2LMHeadModel

tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>',
                                                    eos_token='</s>',
                                                    unk_token='<unk>',
                                                    pad_token='<pad>',
                                                    mask_token='<mask>')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

# inference function

In [5]:
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

In [6]:
from sklearn.metrics.pairwise import pairwise_distances
embedding_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

def cosine_score(input_text, generated_text):
    input_embedding = embedding_model.encode(input_text)
    generated_embedding = embedding_model.encode(generated_text)

    cosine_similarity_score = cosine_similarity(input_embedding, generated_embedding)
    cosine_similarity_score = max(cosine_similarity_score, 0)

    return cosine_similarity_score

def evaluate_similarity(input_text, generated_text, alpha=0.95): # cosine 가중치 조절
    input_embedding = embedding_model.encode(input_text)
    generated_embedding = embedding_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [7]:
def post_processing(text):
    import re
    end = text.rfind('</s>')
    if end:
        text = text[:end]
    text = text.strip()
    text = text.replace('\n', '')
    text = re.sub(' +', ' ', text)
    return text

In [8]:
def ValidChatbot(input_text, sim_answer, model, tokenizer=tokenizer, device=device,
            max_length=200, temperature=0.87, top_k=27, top_p=0.7, num_samples=5, generated=True):

    model.eval()
    input_text = input_text.strip().replace('"', '')
    sim_answer = sim_answer.strip().replace('"', '')
    text = '<q>' + input_text + '</s><a>'
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
    q_len = len(text) + 1

    best_generated_text = None
    best_similarity_score = -1.0
    score_lst = []
    generated_texts = []
    for i in range(num_samples):
        result_ids = model.generate(input_ids,
                                    max_length=max_length,
                                    temperature=temperature,
                                    top_k=top_k,
                                    top_p=top_p,
                                    do_sample=True,
                                    num_return_sequences=1,
                                    )

        generated_text = tokenizer.decode(result_ids[0])
        generated_text = post_processing(generated_text[q_len:])

        similarity_score = evaluate_similarity(sim_answer, generated_text)
        score_lst.append(similarity_score)
        generated_texts.append((similarity_score,generated_text))
        if generated:
            print(generated_texts[i]) # 실제 돌릴땐 off

        if similarity_score > best_similarity_score:
            best_similarity_score = similarity_score
            best_generated_text = generated_text

    return best_generated_text, score_lst

# 비교

## 검증셋으로 추론

In [9]:
valid_sim = pd.read_csv(f'{DATA_PATH}valid_sim.csv')
valid_sim

,질문,답변_1,유사_답변,유사도
0,테라죠의 평균 수명은 몇 년인가요?,테라죠의 기대수명은 30년입니다.,테라죠의 기대수명은 30년입니다.,[0.9864692687988281]
1,플로킹 벽지란 무엇을 의미하나요?,벨벳 소재를 사용하여 입체감을 살린 벽지입니다. 고급스러운 분위기를 연출하며 포인트...,벨벳 소재를 사용하여 입체감을 살린 벽지입니다. 고급스러운 분위기를 연출하며 포인트...,[0.9820297360420227]
2,건조는 며칠 동안 진행되어야 하나요?,도배 후 도배지가 완전히 건조되기까지 최대 일주일이 걸릴 수도 있습니다.,도배 후 도배지가 완전히 건조되기까지 최대 일주일이 걸릴 수도 있습니다.,[0.8654287457466125]
3,장판의 장점이 뭔가요?,장판은 단가와 시공비가 저렴하고 보행감이 좋으며 높은 열전도율로 난방효과가 좋고 층...,장판은 단가와 시공비가 저렴하고 보행감이 좋으며 높은 열전도율로 난방효과가 좋고 층...,[0.9892034530639648]
4,도막방수공사는 어떤 작업인가요?,"도막방수는 방수용으로 제조된 우레탄 고무계, 아크릴 고무계, 고무 아스팔트계 등의 ...","도막방수는 방수용으로 제조된 우레탄 고무계, 아크릴 고무계, 고무 아스팔트계 등의 ...",[0.9870628714561462]
...,...,...,...,...
95,"발수공사란 무엇을 무엇이고, 천연 재료로 만들어진 벽지에 대해 설명해주세요.","발수공사란 콘크리트 구체 및 모르타르, 벽돌 벽체 등 흡수성을 갖는 바탕재에 고분자...","발수공사란 콘크리트 구체 및 모르타르, 벽돌 벽체 등 흡수성을 갖는 바탕재에 고분자...","[0.9544162750244141, 0.785067617893219]"
96,도배 작업에는 몇 명이 필요한가요? 그리고 마루에 생긴 스크래치를 숨길 방법이 있을까요?,"일반적으로 30평 아파트 기준 도배에 필요한 인원은 합지벽지는 3명의 도배사, 실크...","일반적으로 30평 아파트 기준 도배에 필요한 인원은 합지벽지는 3명의 도배사, 실크...","[0.9608222842216492, 0.9883949756622314]"
97,"마루의 장점은 무엇인가요? 또한, 낡은 목재 가구에 있는 흠집을 감춰주는 방법을 알...","마루는 나무본연의 부드러운 촉감과 질감을 느낄 수 있다는 장점이 있습니다. 또한, ...",마루는 나무본연의 부드러운 촉감과 질감을 느낄 수 있다는 장점이 있습니다. 흠집을 ...,"[0.9999998211860657, 0.914232075214386]"
98,"속건형 유성 발수제란 무엇인지 무엇이고, 면진과 제진의 구조에는 어떤 차이가 있나요?",속건형 유성 발수제는 유성계 발수제로서 도장 후 용제가 증발되면서 발수효과를 발휘하...,속건형 유성 발수제는 유성계 발수제로서 도장 후 용제가 증발되면서 발수효과를 발휘하...,"[0.9676459431648254, 0.7238320112228394]"


### 3epoch

In [10]:
model_name = 'kogpt2_120000_3epoch_0.21946loss'

model = GPT2LMHeadModel.from_pretrained(f'{DATA_PATH}model/{model_name}').to(device)
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>',
                                                    eos_token='</s>',
                                                    unk_token='<unk>',
                                                    pad_token='<pad>',
                                                    mask_token='<mask>')

mean_score = []
answer_list = []
for _, row in tqdm(valid_sim.iterrows(), total=valid_sim.shape[0]):
    reset_seeds(SEED)
    text = row['질문']
    similar_text = row['유사_답변']
    answer, score = ValidChatbot(text, similar_text, model=model, tokenizer=tokenizer, generated=False, max_length=200, temperature=0.5, top_k=20, top_p=0.95)
    answer_list.append(answer)
    mean_score.append(score)

scores = [cosine_score(target, pred) for pred, target in zip(answer_list, valid_sim['답변_1'])]
print(np.mean(scores))

  0%|          | 0/100 [00:00<?, ?it/s]

0.91640437


In [11]:
for a, b, c in zip(valid_sim['질문'].tolist(), answer_list, mean_score) :
    print(f'질문 : {a}')
    print(f'답변 : {b}')
    print(f'답변 : {c}')
    print(' ')

질문 : 테라죠의 평균 수명은 몇 년인가요?
답변 : 테라죠의 기대수명은 30년입니다.
답변 : [0.6023155634817869, 0.6059481759866078, 1.0, 1.0, 1.0]
 
질문 : 플로킹 벽지란 무엇을 의미하나요?
답변 : 벨벳 소재를 사용하여 입체감을 살린 벽지입니다. 고급스러운 분위기를 연출하며 포인트 용으로 적합하다는 장점이 있지만 가격이 비싸고 내구성이 떨어지며 관리가 힘들다는 단점이 있습니다.
답변 : [0.7447502803234827, 1.0, 0.7428752160349558, 0.7122567574183146, 0.6558567714691161]
 
질문 : 건조는 며칠 동안 진행되어야 하나요?
답변 : 건조는 주로 건축물이나 시설물 건설 현장에서 사용되는 작업으로, 방청도료가 완전히 건조된 후에 다음 단계를 진행합니다. 건조가 완료된 후에, 도배지가 완전히 건조되기까지 최대 일주일이 걸릴 수도 있습니다. 건조를 마친 후에는 제조사의 권장사항을 준수하여 완전히 건조된 후 페인트를 칠합니다. 건조가 완료된 후에는 페인트를 바를 수 있습니다. 건조가 완료된 후에는 페인트를 바를 수 있습니다. 건조가 완료된 후에는 다시 건조를 할 수 있습니다. 건조가 완료된 후에 페인트를 바를 수 있습니다. 건조가 완료된 후에, 페인트를 바를 수 있습니다.
답변 : [0.30837604036697974, 0.27752258777618405, 0.3373058497905731, 0.5072774923764742, 0.38132801658577387]
 
질문 : 장판의 장점이 뭔가요?
답변 : 장판은 단가와 시공비가 저렴하여 비용 효율적이며 보행감이 좋아 편안한 걷기가 가능합니다. 또한, 높은 열전도율로 인해 난방 효과가 좋고, 층간 소음 방지에도 탁월한 효과를 발휘합니다.
답변 : [0.3371748935235174, 0.3371748935235174, 0.8953340470790863, 0.8953340470790863, 0.80081

### 3epoch(default parameters)

In [ ]:
model_name = 'kogpt2_120000_3epoch_0.21946loss'

model = GPT2LMHeadModel.from_pretrained(f'{DATA_PATH}model/{model_name}').to(device)
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>',
                                                    eos_token='</s>',
                                                    unk_token='<unk>',
                                                    pad_token='<pad>',
                                                    mask_token='<mask>')

mean_score = []
answer_list = []
for _, row in tqdm(valid_sim.iterrows(), total=valid_sim.shape[0]):
    reset_seeds(SEED)
    text = row['질문']
    similar_text = row['유사_답변']
    answer, score = ValidChatbot(text, similar_text, model=model, tokenizer=tokenizer, generated=False) # default
    answer_list.append(answer)
    mean_score.append(score)

scores = [cosine_score(target, pred) for pred, target in zip(answer_list, valid_sim['답변_1'])]
print(np.mean(scores))

  0%|          | 0/100 [00:00<?, ?it/s]

0.91349


In [ ]:
for a, b, c in zip(valid_sim['질문'].tolist(), answer_list, mean_score) :
    print(f'질문 : {a}')
    print(f'답변 : {b}')
    print(f'답변 : {c}')
    print(' ')

질문 : 테라죠의 평균 수명은 몇 년인가요?
답변 : 테라죠의 기대수명은 30년입니다.
답변 : [0.6009074047855708, 0.6059482326110204, 1.0, 1.0, 1.0]
 
질문 : 플로킹 벽지란 무엇을 의미하나요?
답변 : 벨벳 소재를 사용하여 입체감을 살린 벽지입니다. 고급스러운 분위기를 연출하며 포인트 용으로 적합하다는 장점이 있지만 가격이 비싸고 내구성이 떨어지며 관리가 힘들다는 단점이 있습니다.
답변 : [0.7428752160349558, 0.9999999433755875, 0.6994984963665837, 0.7039748260508413, 0.9999999433755875]
 
질문 : 건조는 며칠 동안 진행되어야 하나요?
답변 : 건조는 최소 일주일 동안 이루어져야 합니다. 건조 기간 동안 온도와 습도에 유의하여, 제조사의 권장사항을 준수하는 것이 중요합니다. 만약 일주일 후에도 여전히 문제가 있다면, 시공사에 연락하여 상황을 진단받아야 합니다. 또한, 안전을 위해 적절한 환기 및 습도 관리가 필요합니다.
답변 : [0.47805885076522825, 0.3802105635404587, 0.4021930744773463, 0.3681533575057983, 0.361681703502132]
 
질문 : 장판의 장점이 뭔가요?
답변 : 장판은 단가와 시공비가 저렴하여 비용 효율적이며 보행감이 좋아 편안한 걷기가 가능합니다. 또한, 높은 열전도율로 인해 난방 효과가 좋고, 층간 소음 방지에도 탁월한 효과를 발휘합니다.
답변 : [0.3371748935235174, 0.3371748935235174, 0.8953340470790863, 0.8953340470790863, 0.8008173538220895]
 
질문 : 도막방수공사는 어떤 작업인가요?
답변 : 도막방수는 방수용으로 제조된 우레탄 고무계, 아크릴 고무계, 고무 아스팔트계 등의 액상형의 재료를 바탕에 발라 방수층을 형성하는 공법으로 액체방수를 하고 2차로 시

# 원본

## 검증셋으로 추론

In [ ]:
valid_sim = pd.read_csv(f'{DATA_PATH}valid_sim.csv')
valid_sim

,질문,답변_1,유사_답변,유사도
0,테라죠의 평균 수명은 몇 년인가요?,테라죠의 기대수명은 30년입니다.,테라죠의 기대수명은 30년입니다.,[0.9864692687988281]
1,플로킹 벽지란 무엇을 의미하나요?,벨벳 소재를 사용하여 입체감을 살린 벽지입니다. 고급스러운 분위기를 연출하며 포인트...,벨벳 소재를 사용하여 입체감을 살린 벽지입니다. 고급스러운 분위기를 연출하며 포인트...,[0.9820297360420227]
2,건조는 며칠 동안 진행되어야 하나요?,도배 후 도배지가 완전히 건조되기까지 최대 일주일이 걸릴 수도 있습니다.,도배 후 도배지가 완전히 건조되기까지 최대 일주일이 걸릴 수도 있습니다.,[0.8654287457466125]
3,장판의 장점이 뭔가요?,장판은 단가와 시공비가 저렴하고 보행감이 좋으며 높은 열전도율로 난방효과가 좋고 층...,장판은 단가와 시공비가 저렴하고 보행감이 좋으며 높은 열전도율로 난방효과가 좋고 층...,[0.9892034530639648]
4,도막방수공사는 어떤 작업인가요?,"도막방수는 방수용으로 제조된 우레탄 고무계, 아크릴 고무계, 고무 아스팔트계 등의 ...","도막방수는 방수용으로 제조된 우레탄 고무계, 아크릴 고무계, 고무 아스팔트계 등의 ...",[0.9870628714561462]
...,...,...,...,...
95,"발수공사란 무엇을 무엇이고, 천연 재료로 만들어진 벽지에 대해 설명해주세요.","발수공사란 콘크리트 구체 및 모르타르, 벽돌 벽체 등 흡수성을 갖는 바탕재에 고분자...","발수공사란 콘크리트 구체 및 모르타르, 벽돌 벽체 등 흡수성을 갖는 바탕재에 고분자...","[0.9544162750244141, 0.785067617893219]"
96,도배 작업에는 몇 명이 필요한가요? 그리고 마루에 생긴 스크래치를 숨길 방법이 있을까요?,"일반적으로 30평 아파트 기준 도배에 필요한 인원은 합지벽지는 3명의 도배사, 실크...","일반적으로 30평 아파트 기준 도배에 필요한 인원은 합지벽지는 3명의 도배사, 실크...","[0.9608222842216492, 0.9883949756622314]"
97,"마루의 장점은 무엇인가요? 또한, 낡은 목재 가구에 있는 흠집을 감춰주는 방법을 알...","마루는 나무본연의 부드러운 촉감과 질감을 느낄 수 있다는 장점이 있습니다. 또한, ...",마루는 나무본연의 부드러운 촉감과 질감을 느낄 수 있다는 장점이 있습니다. 흠집을 ...,"[0.9999998211860657, 0.914232075214386]"
98,"속건형 유성 발수제란 무엇인지 무엇이고, 면진과 제진의 구조에는 어떤 차이가 있나요?",속건형 유성 발수제는 유성계 발수제로서 도장 후 용제가 증발되면서 발수효과를 발휘하...,속건형 유성 발수제는 유성계 발수제로서 도장 후 용제가 증발되면서 발수효과를 발휘하...,"[0.9676459431648254, 0.7238320112228394]"


### 3epoch

In [ ]:
model_name = 'kogpt2_120000_3epoch_0.21946loss'

model = GPT2LMHeadModel.from_pretrained(f'{DATA_PATH}model/{model_name}').to(device)
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>',
                                                    eos_token='</s>',
                                                    unk_token='<unk>',
                                                    pad_token='<pad>',
                                                    mask_token='<mask>')

mean_score = []
answer_list = []
for _, row in tqdm(valid_sim.iterrows(), total=valid_sim.shape[0]):
    reset_seeds(SEED)
    text = row['질문']
    similar_text = row['유사_답변']
    answer, score = ValidChatbot(text, similar_text, model=model, tokenizer=tokenizer, generated=False, max_length=200, temperature=0.5, top_k=20, top_p=0.95)
    answer_list.append(answer)
    mean_score.append(score)

scores = [cosine_score(target, pred) for pred, target in zip(answer_list, valid_sim['답변_1'])]
print(np.mean(scores))

  0%|          | 0/100 [00:00<?, ?it/s]

0.91640437


In [ ]:
for a, b, c in zip(valid_sim['질문'].tolist(), answer_list, mean_score) :
    print(f'질문 : {a}')
    print(f'답변 : {b}')
    print(f'답변 : {c}')
    print(' ')

질문 : 테라죠의 평균 수명은 몇 년인가요?
답변 : 테라죠의 기대수명은 30년입니다.
답변 : [0.6023155634817869, 0.6059482326110204, 1.0, 1.0, 1.0]
 
질문 : 플로킹 벽지란 무엇을 의미하나요?
답변 : 벨벳 소재를 사용하여 입체감을 살린 벽지입니다. 고급스러운 분위기를 연출하며 포인트 용으로 적합하다는 장점이 있지만 가격이 비싸고 내구성이 떨어지며 관리가 힘들다는 단점이 있습니다.
답변 : [0.7447502803234827, 0.9999999433755875, 0.7428752160349558, 0.7122567574183146, 0.6558567714691161]
 
질문 : 건조는 며칠 동안 진행되어야 하나요?
답변 : 건조는 주로 건축물이나 시설물 건설 현장에서 사용되는 작업으로, 방청도료가 완전히 건조된 후에 다음 단계를 진행합니다. 건조가 완료된 후에, 도배지가 완전히 건조되기까지 최대 일주일이 걸릴 수도 있습니다. 건조를 마친 후에는 제조사의 권장사항을 준수하여 완전히 건조된 후 페인트를 칠합니다. 건조가 완료된 후에는 페인트를 바를 수 있습니다. 건조가 완료된 후에는 페인트를 바를 수 있습니다. 건조가 완료된 후에는 다시 건조를 할 수 있습니다. 건조가 완료된 후에 페인트를 바를 수 있습니다. 건조가 완료된 후에, 페인트를 바를 수 있습니다.
답변 : [0.30837604036697974, 0.27752258777618405, 0.3373058497905731, 0.5072774923764742, 0.38132801658577387]
 
질문 : 장판의 장점이 뭔가요?
답변 : 장판은 단가와 시공비가 저렴하여 비용 효율적이며 보행감이 좋아 편안한 걷기가 가능합니다. 또한, 높은 열전도율로 인해 난방 효과가 좋고, 층간 소음 방지에도 탁월한 효과를 발휘합니다.
답변 : [0.3371748935235174, 0.3371748935235174, 0.8953340470790863, 0.8953340470

### 3epoch(default parameters)

In [ ]:
model_name = 'kogpt2_120000_3epoch_0.21946loss'

model = GPT2LMHeadModel.from_pretrained(f'{DATA_PATH}model/{model_name}').to(device)
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>',
                                                    eos_token='</s>',
                                                    unk_token='<unk>',
                                                    pad_token='<pad>',
                                                    mask_token='<mask>')

mean_score = []
answer_list = []
for _, row in tqdm(valid_sim.iterrows(), total=valid_sim.shape[0]):
    reset_seeds(SEED)
    text = row['질문']
    similar_text = row['유사_답변']
    answer, score = ValidChatbot(text, similar_text, model=model, tokenizer=tokenizer, generated=False) # default
    answer_list.append(answer)
    mean_score.append(score)

scores = [cosine_score(target, pred) for pred, target in zip(answer_list, valid_sim['답변_1'])]
print(np.mean(scores))

  0%|          | 0/100 [00:00<?, ?it/s]

0.91349


In [ ]:
for a, b, c in zip(valid_sim['질문'].tolist(), answer_list, mean_score) :
    print(f'질문 : {a}')
    print(f'답변 : {b}')
    print(f'답변 : {c}')
    print(' ')

질문 : 테라죠의 평균 수명은 몇 년인가요?
답변 : 테라죠의 기대수명은 30년입니다.
답변 : [0.6009074047855708, 0.6059482326110204, 1.0, 1.0, 1.0]
 
질문 : 플로킹 벽지란 무엇을 의미하나요?
답변 : 벨벳 소재를 사용하여 입체감을 살린 벽지입니다. 고급스러운 분위기를 연출하며 포인트 용으로 적합하다는 장점이 있지만 가격이 비싸고 내구성이 떨어지며 관리가 힘들다는 단점이 있습니다.
답변 : [0.7428752160349558, 0.9999999433755875, 0.6994984963665837, 0.7039748260508413, 0.9999999433755875]
 
질문 : 건조는 며칠 동안 진행되어야 하나요?
답변 : 건조는 최소 일주일 동안 이루어져야 합니다. 건조 기간 동안 온도와 습도에 유의하여, 제조사의 권장사항을 준수하는 것이 중요합니다. 만약 일주일 후에도 여전히 문제가 있다면, 시공사에 연락하여 상황을 진단받아야 합니다. 또한, 안전을 위해 적절한 환기 및 습도 관리가 필요합니다.
답변 : [0.47805885076522825, 0.3802105635404587, 0.4021930744773463, 0.3681533575057983, 0.361681703502132]
 
질문 : 장판의 장점이 뭔가요?
답변 : 장판은 단가와 시공비가 저렴하여 비용 효율적이며 보행감이 좋아 편안한 걷기가 가능합니다. 또한, 높은 열전도율로 인해 난방 효과가 좋고, 층간 소음 방지에도 탁월한 효과를 발휘합니다.
답변 : [0.3371748935235174, 0.3371748935235174, 0.8953340470790863, 0.8953340470790863, 0.8008173538220895]
 
질문 : 도막방수공사는 어떤 작업인가요?
답변 : 도막방수는 방수용으로 제조된 우레탄 고무계, 아크릴 고무계, 고무 아스팔트계 등의 액상형의 재료를 바탕에 발라 방수층을 형성하는 공법으로 액체방수를 하고 2차로 시

# 실제 테스트

In [24]:
test = pd.read_csv(f'{DATA_PATH}test_sim.csv')
test

,id,질문,유사_답변
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용...","방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?,해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹...
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?",큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 ...
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있...","단열재의 종류는 1. 압출법 보온판, 2. 비드법 보온판 1종, 3. 비드법 보온판..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?,도배 후 도배지가 완전히 건조되기까지 최대 일주일이 걸릴 수도 있습니다.
...,...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...,"분말 소화기를 사용할 때 주의할 점은 압력계의 눈금 위치가 초록색에 있는지 확인, ..."
126,TEST_126,압출법 보온판의 가장 큰 장점은 무엇인가요?,압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전...
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...,수성 벽체용 탄성 방수 도료는 치밀한 표면구성 및 탄성있는 도막을 형성하여 방수성능...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...,벽을 이루고 있는 석고에 훼손이 있는 경우를 석고수정이라고 합니다. 석고수정이 발생...


In [21]:
def Chatbot(input_text, sim_answer, model=model, tokenizer=tokenizer, device=device,
            max_length=200, temperature=0.87, top_k=27, top_p=0.7, num_samples=5, generated=True):

    model.eval()
    input_text = input_text.strip().replace('"', '')
    text = '<q>' + input_text + '</s><a>'
    sim_answer = sim_answer.strip().replace('"', '')
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
    q_len = len(text) + 1

    best_generated_text = None
    best_similarity_score = -1.0
    score_lst = []
    generated_texts = []
    for i in range(num_samples):
        result_ids = model.generate(input_ids,
                                    max_length=max_length,
                                    temperature=temperature,
                                    top_k=top_k,
                                    top_p=top_p,
                                    do_sample=True,
                                    num_return_sequences=1,
                                    )

        generated_text = tokenizer.decode(result_ids[0])
        generated_text = post_processing(generated_text[q_len:])

        similarity_score = evaluate_similarity(sim_answer, generated_text)
        score_lst.append(similarity_score)
        generated_texts.append((similarity_score,generated_text))
        if generated:
            print(generated_texts[i]) # 실제 돌릴땐 off

        if similarity_score > best_similarity_score:
            best_similarity_score = similarity_score
            best_generated_text = generated_text

    return best_generated_text, score_lst

In [25]:
model_name = 'kogpt2_120000_3epoch_0.21946loss'

model = GPT2LMHeadModel.from_pretrained(f'{DATA_PATH}model/{model_name}').to(device)
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>',
                                                    eos_token='</s>',
                                                    unk_token='<unk>',
                                                    pad_token='<pad>',
                                                    mask_token='<mask>')

mean_score = []
answer_list = []
for _, row in tqdm(test.iterrows(), total=test.shape[0]):
    reset_seeds(SEED)
    text = row['질문']
    similar_text = row['유사_답변']
    answer, score = Chatbot(text, similar_text, model=model, tokenizer=tokenizer, generated=False) # default
    answer_list.append(answer)
    mean_score.append(score)

  0%|          | 0/130 [00:00<?, ?it/s]

In [26]:
for a, b, c in zip(test['질문'].tolist(), answer_list, mean_score) :
    print(f'질문 : {a}')
    print(f'답변 : {b}')
    print(f'답변 : {c}')
    print(' ')

질문 : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
답변 : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 경우 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점이 단점입니다.
답변 : [0.9672923158253393, 0.7115952700376511, 0.9672923158253393, 0.9672923158253393, 0.7039061401472535]
 
질문 : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
답변 : 해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상으로, 다음과 같은 원인들이 있습니다. 첫째, 높은 습도로 인해 도배지 안쪽의 금속이 녹을 수 있습니다. 이 경우 건물 소유자나 거주자는 습기 관리에 책임이 있으며, 제습기를 사용하거나 환기를 통해 실내 습도를 조절해야 합니다. 전문가의 도움을 받아 보수 작업을 해야 합니다. 둘째, 누수로 인해 도배지가 젖어 있는 상태가 계속되면 곰팡이와 녹이 생길 수 있습니다. 이 경우 건물 소유자나 거주자는 책임이 있으며, 누수를 제거하고 곰팡이와 녹이 발생한 부분을 교체해야 합니다. 전문가의 도움을 받는 것이 좋습니다. 마지막으로, 누수로 인한 도배지의 녹이 발생한 경우 건물 소유자나 거주자는 책임이 있으며, 누수를 제거하고 곰팡이와 녹이 발생한 부분을 교체해야 합니다. 이러한 작업은 전문가의 도움이 필요합니다. 따라서, 상황에 따라 적합한 적절한 대책을 선택하여 해당 현상을 해결하는 것이 중요합니다.
답변 : [0.7027739047780073, 0.6733085551914179, 0.7176767690564101, 0.8670782109586204, 0.7015354629485837]
 
질문 : 큐블럭의 단점을 알려주세요

# submission

In [27]:
vector_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
pred_vector = vector_model.encode(answer_list)
pred_vector.shape

(130, 512)

In [28]:
submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
submission.iloc[:,1:] = pred_vector
submission

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.033631,-0.014400,0.014687,0.012787,0.073650,0.019086,0.002030,-0.010057,-0.019762,...,-0.014742,-0.002076,-0.028723,-0.035333,-0.002346,0.018736,0.033766,0.033795,0.009566,0.012993
1,TEST_001,-0.019230,-0.026600,0.025928,0.016634,0.075547,-0.007247,-0.021234,0.028983,0.020994,...,-0.020339,-0.023694,0.037386,-0.046874,-0.051016,0.036335,-0.001647,-0.041393,0.052818,0.006819
2,TEST_002,0.026435,-0.051316,-0.023122,-0.005046,0.123666,-0.052146,-0.025211,-0.028712,0.042899,...,-0.038931,-0.013700,0.030878,-0.022841,0.018967,0.011641,-0.038178,-0.041414,-0.049640,0.092450
3,TEST_003,0.020456,0.014602,-0.005448,0.021697,0.066295,-0.049532,-0.060348,-0.018360,-0.011505,...,-0.044512,0.017548,0.052905,-0.040040,0.010618,0.013271,-0.044331,-0.028638,0.021200,0.021273
4,TEST_004,0.009517,0.028026,0.005988,-0.025771,0.093989,-0.003675,-0.007816,0.115160,-0.019709,...,-0.016562,-0.010899,0.051777,0.052240,0.024490,0.048989,-0.019329,-0.008300,0.000251,-0.034930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,TEST_125,0.030860,-0.044582,0.027274,0.015991,0.117427,-0.067691,0.066928,0.042426,0.054038,...,0.015964,0.010538,0.047512,-0.039530,0.021517,0.048241,0.044327,-0.024992,-0.032482,0.007421
126,TEST_126,0.048907,-0.011243,-0.043629,0.029416,0.083383,-0.053422,-0.033635,0.029032,0.007985,...,-0.042395,-0.064186,-0.000828,-0.053311,0.057252,0.006373,-0.021555,-0.051903,-0.021566,0.044492
127,TEST_127,-0.002169,0.025316,0.008737,0.020653,0.060620,-0.009253,-0.000711,-0.017798,-0.033897,...,-0.026501,-0.015872,-0.008425,-0.027854,-0.023853,0.027221,-0.012905,0.036481,-0.042230,0.024975
128,TEST_128,0.036386,-0.027929,-0.063045,0.015607,0.084918,-0.076239,0.011936,0.024018,-0.014002,...,-0.026435,-0.025367,0.003164,-0.060882,-0.018098,0.008412,0.012525,-0.015694,0.026506,0.010083


In [29]:
submission.to_csv(f'{DATA_PATH}hansol_120000_3epoch_rule_based_AdamW_lr_scheduler_params_2.csv', index=False)